# Learnig the representations

In [1]:
# Dependecies
import importlib
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# device agnostic code
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## pasted models
here for convenience

In [21]:
'''
    Representation learning:
    Goal: use unsupervised learning techniques to learn a representation of given data.
    The hope is that this representation will be useful to reduce the amount of data that is needed for training the supervised model for solving the actual task.
    To verify how good the learned representation is, train a supervised model using these representations that predicts the available pretrain labels.

    Methology:
    1. Create a several neural networks that learn to encode the data into a representation.
    2. Train a supervised model on each of the learned representations. The superverised model trained on the different representations should be very shallow (1 or two fully connected layers) and should be trained for a very short time. The goal is to make the performance of the encoders comparable.
'''

# Dependencies
import torch
import torch.nn as nn

'''
    Autoencoder for dimensionality reduction:
    Both encoder and decoder using three linear layers
'''

# for this to make sense the encoding dimension should be significantly smaller than the input dimension
# specifically, the encoding_dim*3 shold be smaller than the input_size
class LinearAutoencoder(nn.Module):
    def __init__(self, input_size, encoding_dim):
        super(LinearAutoencoder, self).__init__()
        # encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_size, encoding_dim*3),
            nn.ReLU(),
            nn.Linear(int(encoding_dim*3), int(encoding_dim*2)),
            nn.ReLU(),
            nn.Linear(encoding_dim*2, encoding_dim)
        )
        # decoder
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, encoding_dim*2),
            nn.ReLU(),
            nn.Linear(encoding_dim*2, encoding_dim*3),
            nn.ReLU(),
            nn.Linear(encoding_dim*3, input_size),
            nn.Sigmoid() # the feature values are between 0 and 1
        )
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


'''
    Autocoder for dimensionality reduction:
    Using three convolutional/deconvolutional layers for encoder/decoder
'''
class ConvAutoencoder(nn.Module):
    def __init__(self, number_filters):
        super(ConvAutoencoder, self).__init__()
        # encoder
        self.encoder = nn.Sequential(
            nn.Conv1d(1, number_filters*3, kernel_size=3, stride=2, padding=0),
            nn.ReLU(),
            nn.Conv1d(number_filters*3, number_filters*2, kernel_size=3, stride=2, padding=0),
            nn.ReLU(),
            nn.Conv1d(number_filters*2, number_filters, kernel_size=3, stride=2, padding=0)
        )
        # decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(number_filters, number_filters*2, kernel_size=3, stride=2, padding=0),
            nn.ReLU(),
            nn.ConvTranspose1d(number_filters*2, number_filters*3, kernel_size=3, stride=2, padding=0),
            nn.ReLU(),
            nn.ConvTranspose1d(number_filters*3, 1, kernel_size=3, stride=2, padding=0, output_padding=1), # need out padding to get the right size
            nn.Sigmoid() # the feature values are between 0 and 1
        )
    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.encoder(x)
        x = self.decoder(x)
        x = x.squeeze(1)
        return x
    
'''
    Autocoder for dimensionality reduction:
    Using two convolutional/deconvolutional layers and one fully connected layer for both encoder and decoder
'''
class ConvLinearAutoencoder(nn.Module):
    def __init__(self, number_filters, encoding_dim):
        super(ConvLinearAutoencoder, self).__init__()
        self.number_filters = number_filters
        # encoder
        self.encoder = nn.Sequential(
            nn.Conv1d(1, number_filters*2, kernel_size=3, stride=2, padding=0),
            nn.ReLU(),
            nn.Conv1d(number_filters*2, number_filters, kernel_size=3, stride=2, padding=0),
            nn.ReLU(),
        ) 
        # bottleneck layer
        self.fcencoder = nn.Linear(249*number_filters, encoding_dim)
        self.fcdecoder = nn.Linear(encoding_dim, 249*number_filters)

        # decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(number_filters, number_filters*2, kernel_size=3, stride=2, padding=0),
            nn.ReLU(),
            nn.ConvTranspose1d(number_filters*2, 1, kernel_size=3, stride=2, padding=0, output_padding=1),
            nn.Sigmoid() # the feature values are between 0 and 1
        )
    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.encoder(x)
        x = x.view(-1, 249*self.number_filters)
        x = self.fcencoder(x)
        x = self.fcdecoder(x)
        x = x.view(-1,self.number_filters, 249)
        x = self.decoder(x)
        x = x.squeeze(1)
        return x
    
# contractive loss function
def contractive_loss(W, x, recons_x, h, lam=1e-4):
    mse_loss = nn.MSELoss()(recons_x, x)
    
    dh = h * (1 - h) # Derivative of sigmoid
    w_sum = torch.sum(Variable(W)**2, dim=1)
    w_sum = w_sum.unsqueeze(1) # Shape to 2D tensor
    contractive_loss = torch.sum(torch.mm(dh**2, w_sum), 0)
    return mse_loss + contractive_loss.mul_(lam)

## load data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import shutil
shutil.unpack_archive("drive/MyDrive/SolarEnergyMaterials/task4.zip", "/content/data")
shutil.unpack_archive("data/task4_hr35z9/pretrain_features.csv.zip", "/content/data")
shutil.unpack_archive("data/task4_hr35z9/pretrain_labels.csv.zip", "/content/data")
shutil.unpack_archive("data/task4_hr35z9/train_features.csv.zip", "/content/data")
shutil.unpack_archive("data/task4_hr35z9/train_labels.csv.zip", "/content/data")

In [6]:
def load_pretrain_data(batch_size = 64):
    batch_size = 64

    random.seed(17)
    test_ind = set()

    pre_train_size = 50000

    while len(test_ind) < 10000: 
        test_ind.add(random.randint(0, pre_train_size-1))

    features =[]
    labels = []

    with open("data/pretrain_features.csv", 'r') as f:
        for row in f:
            features.append(row)

    with open("data/pretrain_labels.csv", 'r') as f:
        for row in f:
            labels.append(row)

    # remove header
    features = features[1:]
    labels = labels[1:]

    # first try to note use representation of the molecules, only the extracted features
    features = [list(map(float,row.split(',')[2:])) for row in features]
    labels = [float(row.split(',')[1]) for row in labels]

    train_features = []
    train_labels = []
    test_features = []
    test_labels = []


    for i in range(len(features)):
        if i in test_ind:
            test_features.append(features[i])
            test_labels.append(labels[i])
        else:
            train_features.append(features[i])
            train_labels.append(labels[i])

    # does not seem to make sense to normalize the data since it is very sparse
    # normalize train_features
    # train_features = (train_features - np.mean(train_features, axis=0)) / (np.std(train_features, axis=0)+EPSILON)

    # normalize test_features
    # test_features = (test_features - np.mean(test_features, axis=0)) / (np.std(test_features, axis=0)+EPSILON)

    # convert into tensor dataset
    train_features = torch.tensor(train_features, dtype=torch.float)
    train_labels = torch.tensor(train_labels, dtype=torch.float)
    test_features = torch.tensor(test_features, dtype=torch.float)
    test_labels = torch.tensor(test_labels, dtype=torch.float)

    train_dataset = torch.utils.data.TensorDataset(train_features, train_labels)
    test_dataset = torch.utils.data.TensorDataset(test_features, test_labels) 
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

In [7]:
train_loader, test_loader = load_pretrain_data(batch_size = 64)

## Train/Test loop

In [8]:
# train loop:
def train_encoder(model, dataloader, epochs):
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.MSELoss()

    for epoch in range(epochs):
        epoch_loss=0
        for batch, (X, y) in enumerate(dataloader):
            optimizer.zero_grad()
            X_pred = model(X)
            loss = loss_fn(X_pred, X)
            epoch_loss+=loss.item()
            loss.backward()  
            optimizer.step()

        print('average loss per batch in epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, epoch_loss/len(dataloader)))

## Linear Autoencoder:

In [9]:
LinearEncoder = LinearAutoencoder(1000, 128)
train_encoder(LinearEncoder, train_loader, 10)

average loss per batch in epoch [1/10], Loss: 0.0399
average loss per batch in epoch [2/10], Loss: 0.0225
average loss per batch in epoch [3/10], Loss: 0.0155
average loss per batch in epoch [4/10], Loss: 0.0122
average loss per batch in epoch [5/10], Loss: 0.0099
average loss per batch in epoch [6/10], Loss: 0.0084
average loss per batch in epoch [7/10], Loss: 0.0074
average loss per batch in epoch [8/10], Loss: 0.0067
average loss per batch in epoch [9/10], Loss: 0.0061
average loss per batch in epoch [10/10], Loss: 0.0057


In [13]:
torch.save(LinearEncoder.state_dict(), 'drive/MyDrive/SolarEnergyMaterials/PretrainedModels/LinearEncoder.pth')

## Convolutional Autoencoder:

In [10]:
ConvEncoder = ConvAutoencoder(4)
test = next(iter(train_loader))
train_encoder(ConvEncoder, train_loader, 10)

average loss per batch in epoch [1/10], Loss: 0.0712
average loss per batch in epoch [2/10], Loss: 0.0305
average loss per batch in epoch [3/10], Loss: 0.0091
average loss per batch in epoch [4/10], Loss: 0.0030
average loss per batch in epoch [5/10], Loss: 0.0023
average loss per batch in epoch [6/10], Loss: 0.0021
average loss per batch in epoch [7/10], Loss: 0.0020
average loss per batch in epoch [8/10], Loss: 0.0018
average loss per batch in epoch [9/10], Loss: 0.0017
average loss per batch in epoch [10/10], Loss: 0.0015


In [14]:
torch.save(ConvEncoder.state_dict(), 'drive/MyDrive/SolarEnergyMaterials/PretrainedModels/ConvEncoder.pth')

## Convolutional Autoencoder with Linear Layer:

In [22]:
ConvLinearEncoder = ConvLinearAutoencoder(6, 90)
train_encoder(ConvLinearEncoder, train_loader, 10)

average loss per batch in epoch [1/10], Loss: 0.0248
average loss per batch in epoch [2/10], Loss: 0.0081
average loss per batch in epoch [3/10], Loss: 0.0068
average loss per batch in epoch [4/10], Loss: 0.0063
average loss per batch in epoch [5/10], Loss: 0.0060
average loss per batch in epoch [6/10], Loss: 0.0058
average loss per batch in epoch [7/10], Loss: 0.0057
average loss per batch in epoch [8/10], Loss: 0.0056
average loss per batch in epoch [9/10], Loss: 0.0055
average loss per batch in epoch [10/10], Loss: 0.0054


In [23]:
torch.save(ConvEncoder.state_dict(), 'drive/MyDrive/SolarEnergyMaterials/PretrainedModels/ConvEncoder.pth')

## Evaluating the representations
With the pretrain data

In [30]:
def freeze_weights(model):
    for param in model.parameters():
        param.requires_grad = False
